In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls
!mkdir images
%cd images

drive  sample_data
/content/images


In [ ]:
%cp -av "/content/drive/My Drive/master_project/images.rar" "/content/images"


'/content/drive/My Drive/master_project/images.rar' -> '/content/images/images.rar'


In [ ]:
%capture
!unrar x images.rar 
!rm images.rar


In [ ]:
import	numpy  as  np
import	cv2
from	matplotlib  import pyplot  as	plt
import os
import  sys
from google.colab.patches import cv2_imshow
from multiprocessing import Pool

import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**31,-1))
sys.setrecursionlimit(10**9)

In [ ]:
def check_flower(img, thresh, height, width):
    visted = set()
    area = []
    for y, row in enumerate(img):
        for x, val in enumerate(row):
            if (x,y) in visted:
                continue
            if val >= thresh:
                points = set()
                bounds = [width,height,0,0]
                image_bfs(img,thresh,x,y,height, width, points, bounds)
                visted |= points
                if (bounds[2]-bounds[0])*(bounds[3]-bounds[1]) > height*width*0.10:
                    area.append((*bounds, len(points),len(points)*1./((bounds[2]-bounds[0])*(bounds[3]-bounds[1])),(bounds[2]-bounds[0])*(bounds[3]-bounds[1])))
    return area

def image_bfs(img, thresh, x, y, height, width, points, bound):
    if (x,y) in points or img[y][x] < thresh:
        
        return
    points.add((x,y))
    bound[0] = min(x,bound[0])
    bound[1] = min(y,bound[1])
    bound[2] = max(x,bound[2])
    bound[3] = max(y,bound[3])
    if x > 0 and (x-1, y) not in points:
        image_bfs(img, thresh, x-1, y, height, width, points, bound)
    if y > 0 and (x, y-1) not in points:
        image_bfs(img, thresh, x, y-1, height, width, points, bound)
    if x < width-1 and (x+1, y) not in points:
        image_bfs(img, thresh, x+1, y, height, width, points, bound)
    if y < height-1 and (x, y+1) not in points:
        image_bfs(img, thresh, x, y+1, height, width, points, bound)

def area_check(area,i1,i2):
    x_l = max(area[i1][0],area[i2][0])
    y_t = max(area[i1][1],area[i2][1])
    x_r = min(area[i1][2],area[i2][2])
    y_b = min(area[i1][3],area[i2][3])
    if y_t > y_b or x_l > x_r:
        return None
    # print('area check',(y_b-y_t)*(x_r-x_l),area[i1][4],area[i2][4])
    if (y_b-y_t)*(x_r-x_l) <= 0.6*min(area[i1][6],area[i2][6]):
        return None
    elif (y_b-y_t)*(x_r-x_l) >= min(area[i1][6],area[i2][6]):
        return i1 if area[i1][4]<area[i2][4] else i2
    else:
        return i1 if area[i1][5]<area[i2][5] else i2

def image_segment(path, target_path,thresh = 128):
    # print('{:.2f} MB'.format(os.stat(path).st_size/(2**20)))

    if os.stat(path).st_size > 1024*1.5*1024:
        # print('too large')
        return
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # print('shape',img.shape)
    height, width, _ = img.shape
    if height>=1500 or width >= 1500:
        # print('size too large')
        return 
    # print(os.getpid(),path)
    res = []
    # cv2_imshow(img)
    for i in range(3):
        res.extend(check_flower(img[:,:,i].tolist(),thresh,height, width))
    # print(res)
    passed = set()
    for ind,crop in enumerate(res):
        if ind in passed:
            continue
        for i in range(ind+1,len(res)):
            # print(area_check(res,ind,i))
            passed.add(area_check(res,ind,i))
        if ind in passed:
            continue
        # cv2_imshow(img[crop[1]:crop[3],crop[0]:crop[2]])
        file_name = target_path + '{}'.format('_'.join(map(str,crop[:4]))) + '_' + path.split('/')[-1]
        # print(file_name)
        cv2.imwrite(file_name,img[crop[1]:crop[3],crop[0]:crop[2]])
    # print(passed)

In [ ]:
# !rm -rf "/content/drive/My Drive/master_project/crop_2"

In [ ]:
def seg_folder(folder,target,checked):
    print('start',os.getpid(),folder)
    count = 0
    for img in os.listdir(folder):
        try:
            if img in checked:
                continue
            image_segment(os.path.join(folder,img),target)
            count += 1
        except Exception as e:
            print(os.getpid(),e)
            continue
    print('finished',os.getpid(),folder,count)

In [ ]:
path = '/content/images/'
types = os.listdir(path)
types.sort()
dist = '/content/drive/My Drive/master_project/crop_3/'
p1 = '/content/drive/My Drive/master_project/crop/'
if not os.path.isdir(dist):
    os.mkdir(dist)
print(types)
p = Pool()
for flower in types:
    if not os.path.isdir(dist+flower):
        os.mkdir(dist+flower)
    l = os.listdir(dist+flower+'/')
    l.extend(os.listdir(p1+flower+'/'))
    l = map(lambda x: '_'.join(x.split('_')[4:]),l)
    l = set(l)
    try:
        res = p.apply_async(seg_folder,(path+flower+'/',dist+flower+'/',l))
        # seg_folder(path+flower+'/',dist+flower+'/',l)
        
    except Exception as e:
        print(e)
p.close()
p.join()
print('all finished')

In [ ]:
p1 = '/content/drive/My Drive/master_project/crop/'
p3 = '/content/drive/My Drive/master_project/crop_2/Shiyan/'
images = {}
for t in os.listdir(p3):
    try:
        s3 = set(os.listdir(p3+t+'/'))
        s1 = set(os.listdir(p1+t+'/'))
        print(t)
        print(len(s1))
        print(len(s3))
        print(len(s3&s1))
        count = 0

        for f_name in s3&s1:
            print(f_name)
            # os.remove(p3+t+'/'+f_name)
    except Exception as e:
        print(e)


[Errno 2] No such file or directory: '/content/drive/My Drive/master_project/crop/peony/'
[Errno 2] No such file or directory: '/content/drive/My Drive/master_project/crop/start_jasmine/'
[Errno 2] No such file or directory: '/content/drive/My Drive/master_project/crop/sunflower/'
primula
589
1780
0
rose
202
2422
0
thorn_apple
1118
879
0
water_lily
862
1537
0


In [ ]:
# image_segment('/content/images/frangipani/84.jpg','/content')

In [ ]:
# import zipfile
# zf = None
# for ind,f in enumerate(os.listdir(dist)):
#     if ind%6 == 0:
#         if zf:
#             zf.close()
#         zf = zipfile.ZipFile('drive/My Drive/master_project/crop_{}.zip'.format(ind//6+1), mode='w',allowZip64 = True)
#     zf.write(os.path.join(dist,f))